In [1]:
import os
import shutil
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# File move

In [3]:
reducer = ['mean', 'min', 'max', 'sum']
for reducer in reducer:

  # Define the folders
  input_dir = f"/content/drive/MyDrive/GEE_exports_US_{reducer}"
  output_base_dir = "/content/drive/MyDrive/Lab/BNN/Data/PBCNN/phenology"

  # Only move files with these years
  required_years = [str(y) for y in range(2008, 2024)]

  # Loop through each file
  for filename in os.listdir(input_dir):
      if filename.endswith(".csv"):
          # Remove extension and split by underscore
          basename = os.path.splitext(filename)[0]
          parts = basename.split('_')

          # Get the last part as the year
          year = parts[-1]

          # Check if the year is in the required list
          if year in required_years:
              # Build the destination folder path
              year_folder = os.path.join(output_base_dir, year)
              os.makedirs(year_folder, exist_ok=True)

              # Move the file
              src_path = os.path.join(input_dir, filename)
              dest_path = os.path.join(year_folder, filename)
              shutil.move(src_path, dest_path)
              print(f"Moved {filename} to {year_folder}")

# File merge

In [10]:
def insert_reducer_to_column_names(df, reducer):
    # Loop through each column name
    for col in df.columns:
        # Check if any reducer is not already present in the column name
        if reducer not in col:
            # Find the part of the column name before the final part (like '_p60')
            parts = col.split('_')
            if len(parts) > 2:  # Ensure the format matches 'Senescence_1_tmin_p60'
                # Insert the reducer before the last part (e.g., 'p60')
                parts.insert(-1, reducer)  # This will add the reducer before the last segment
                new_col_name = '_'.join(parts)
                df.rename(columns={col: new_col_name}, inplace=True)

    return df

all_years_data = []

for year in range(2008, 2024):
    ##########################################
    # Process yield data
    corn_yield = pd.read_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/yield/yield_{year}.csv')
    corn_yield = corn_yield[['Year', 'State ANSI', 'County ANSI', 'County', 'Value']]
    corn_yield = corn_yield.dropna(subset=['County ANSI'])
    corn_yield['GEOID'] = corn_yield['State ANSI'].astype(int).astype(str).str.zfill(2) + corn_yield['County ANSI'].astype(int).astype(str).str.zfill(3)
    corn_yield = corn_yield.sort_values(by=['GEOID'])
    corn_yield['GEOID'] = corn_yield['GEOID'].astype(str)

    ##########################################################################################
    # Process other data
    folder_path = f"/content/drive/MyDrive/Lab/BNN/Data/PBCNN/phenology/{year}"
    csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")])

    corn_gee_other = None
    reducer_list = ['mean', 'min', 'max', 'sum']

    for file in csv_files:
        file_path = os.path.join(folder_path, file)

        # Check which reducer is in the file name
        reducer_in_file = next((r for r in reducer_list if r in file), None)

        if reducer_in_file:
            print(f"Reducer '{reducer_in_file}' found in file: {file}")

        corn_gee = pd.read_csv(file_path)

        corn_gee.drop(columns=['system:index', '.geo'], inplace=True)
        # Extract "GEOID" and "NAME" columns
        first_columns = ["GEOID", "NAME"]
        # Get remaining columns excluding the first ones
        remaining_columns = [col for col in corn_gee.columns if col not in first_columns]

        # Combine them in the desired order
        sorted_columns = first_columns + remaining_columns
        # Reorder the DataFrame
        corn_gee = corn_gee[sorted_columns]

        # Use insert_reducer function to add the reducer into column names
        if reducer_in_file:
            corn_gee = insert_reducer_to_column_names(corn_gee, reducer_in_file)

        # # Combine them in the desired order
        # sorted_columns = first_columns + remaining_columns
        # # Reorder the DataFrame
        # corn_gee = corn_gee[sorted_columns]

        corn_gee = corn_gee.sort_values(by=['GEOID'])
        corn_gee['GEOID'] = corn_gee['GEOID'].astype(str)

        if corn_gee_other is None:
            corn_gee_other = corn_gee
        else:
            corn_gee_other = pd.merge(corn_gee_other, corn_gee, on=['GEOID', 'NAME'], how='right')

    # Merge with yield data
    corn_gee_final = pd.merge(corn_gee_other, corn_yield[['GEOID', 'Value']], on='GEOID', how='right')
    corn_gee_final = corn_gee_final.dropna()
    corn_gee_final.insert(0, 'year', year)
    all_years_data.append(corn_gee_final)

final_df = pd.concat(all_years_data, ignore_index=True)

Reducer 'max' found in file: EVI2_max_Greenup_1_MidGreenup_1_2008.csv
Reducer 'max' found in file: EVI2_max_Maturity_1_Peak_1_2008.csv
Reducer 'max' found in file: EVI2_max_MidGreendown_1_Dormancy_1_2008.csv
Reducer 'max' found in file: EVI2_max_MidGreenup_1_Maturity_1_2008.csv
Reducer 'max' found in file: EVI2_max_Peak_1_Senescence_1_2008.csv
Reducer 'max' found in file: EVI2_max_Senescence_1_MidGreendown_1_2008.csv
Reducer 'mean' found in file: EVI2_mean_Greenup_1_MidGreenup_1_2008.csv
Reducer 'mean' found in file: EVI2_mean_Maturity_1_Peak_1_2008.csv
Reducer 'mean' found in file: EVI2_mean_MidGreendown_1_Dormancy_1_2008.csv
Reducer 'mean' found in file: EVI2_mean_MidGreenup_1_Maturity_1_2008.csv
Reducer 'mean' found in file: EVI2_mean_Peak_1_Senescence_1_2008.csv
Reducer 'mean' found in file: EVI2_mean_Senescence_1_MidGreendown_1_2008.csv
Reducer 'min' found in file: EVI2_min_Greenup_1_MidGreenup_1_2008.csv
Reducer 'min' found in file: EVI2_min_Maturity_1_Peak_1_2008.csv
Reducer 'mi

# Historical yield

In [13]:
# Define the range of end years you want to loop through
end_year_list = range(2008, 2024)

# Initialize list to store intermediate DataFrames
all_records = []

for end_year in end_year_list:
    start_year = end_year - 5
    print(f"Averaging years: {list(range(start_year, end_year))}")  # <-- Added print here

    merged_df = None

    for year in range(start_year, end_year):
        # Read and process yield data for the year
        corn_yield = pd.read_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/yield/yield_{year}.csv')
        corn_yield = corn_yield[['Year', 'State ANSI', 'County ANSI', 'County', 'Value']]
        corn_yield = corn_yield.dropna(subset=['County ANSI'])

        corn_yield['GEOID'] = corn_yield['State ANSI'].astype(int).astype(str).str.zfill(2) + \
                              corn_yield['County ANSI'].astype(int).astype(str).str.zfill(3)
        corn_yield = corn_yield.sort_values(by=['GEOID'])
        corn_yield['GEOID'] = corn_yield['GEOID'].astype(str)

        # Rename the 'Value' column to include the year as a suffix
        corn_yield = corn_yield[['GEOID', 'Value']].rename(columns={'Value': f'Yield_{year}'})

        # Merge into the main DataFrame
        if merged_df is None:
            merged_df = corn_yield
        else:
            merged_df = pd.merge(merged_df, corn_yield, on='GEOID', how='outer')

    # Drop rows with any missing values
    merged_df = merged_df.dropna()

    # Calculate average yield across the 5-year period
    yield_columns = [f'Yield_{year}' for year in range(start_year, end_year)]
    merged_df['Hist_Yield'] = merged_df[yield_columns].mean(axis=1)

    # Add the year column
    merged_df['year'] = end_year  # represents last year in 5-year window

    # Keep only relevant columns
    result_df = merged_df[['GEOID', 'year', 'Hist_Yield']]

    # Store the result
    all_records.append(result_df)

# Combine all 5-year average windows
final_df_yield = pd.concat(all_records, ignore_index=True)

# Sort by GEOID and Year
final_df_yield = final_df_yield.sort_values(by=['GEOID', 'year'])
final_df_yield = final_df_yield.dropna()

# Display the result
final_df_yield

Averaging years: [2003, 2004, 2005, 2006, 2007]
Averaging years: [2004, 2005, 2006, 2007, 2008]
Averaging years: [2005, 2006, 2007, 2008, 2009]


<ipython-input-13-966ddb9e2b58>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corn_yield['GEOID'] = corn_yield['State ANSI'].astype(int).astype(str).str.zfill(2) + \


Averaging years: [2006, 2007, 2008, 2009, 2010]
Averaging years: [2007, 2008, 2009, 2010, 2011]
Averaging years: [2008, 2009, 2010, 2011, 2012]
Averaging years: [2009, 2010, 2011, 2012, 2013]
Averaging years: [2010, 2011, 2012, 2013, 2014]
Averaging years: [2011, 2012, 2013, 2014, 2015]
Averaging years: [2012, 2013, 2014, 2015, 2016]
Averaging years: [2013, 2014, 2015, 2016, 2017]
Averaging years: [2014, 2015, 2016, 2017, 2018]
Averaging years: [2015, 2016, 2017, 2018, 2019]
Averaging years: [2016, 2017, 2018, 2019, 2020]
Averaging years: [2017, 2018, 2019, 2020, 2021]
Averaging years: [2018, 2019, 2020, 2021, 2022]


,GEOID,year,Hist_Yield
0,01001,2008,82.40
1,01003,2008,111.60
8147,01003,2014,126.22
9306,01003,2015,129.20
10450,01003,2016,126.26
...,...,...,...
11502,56021,2016,117.06
12522,56021,2017,121.62
1737,56029,2008,137.80
3205,56029,2009,139.20


# Final merge

In [14]:
final_df_ultimate = pd.merge(final_df, final_df_yield, on=['GEOID', 'year'], how='right')
final_df_ultimate=final_df_ultimate.dropna()

In [23]:
# Reorder columns to have 'year', 'GEOID', 'NAME', and 'Hist_Yield' first
columns = ['year', 'GEOID', 'NAME', 'Hist_Yield'] + [col for col in final_df_ultimate.columns if col not in ['year', 'GEOID', 'NAME', 'Hist_Yield']]

# Reapply the column order
final_df_ultimate = final_df_ultimate[columns]
# Sort by 'GEOID' and 'year'
final_df_ultimate = final_df_ultimate.sort_values(by=['year', 'GEOID'])
# Display the final DataFrame
final_df_ultimate

,year,GEOID,NAME,Hist_Yield,Greenup_1_EVI2_max_p1,Greenup_1_EVI2_max_p10,Greenup_1_EVI2_max_p20,Greenup_1_EVI2_max_p30,Greenup_1_EVI2_max_p40,Greenup_1_EVI2_max_p50,...,Senescence_1_vpdmin_mean_p20,Senescence_1_vpdmin_mean_p30,Senescence_1_vpdmin_mean_p40,Senescence_1_vpdmin_mean_p50,Senescence_1_vpdmin_mean_p60,Senescence_1_vpdmin_mean_p70,Senescence_1_vpdmin_mean_p80,Senescence_1_vpdmin_mean_p90,Senescence_1_vpdmin_mean_p99,Value
0,2008,17001,Adams,157.20,0.272601,0.336852,0.354639,0.366308,0.374115,0.383987,...,0.570383,0.633873,0.725289,0.805795,0.868289,0.898833,0.960402,1.055823,1.272927,171.0
13,2008,17003,Alexander,158.00,0.236816,0.305964,0.341281,0.362254,0.376021,0.387733,...,0.840587,0.851857,0.873579,0.889186,0.896536,0.907469,0.928081,1.025655,1.099423,144.0
17,2008,17005,Bond,129.80,0.309785,0.364315,0.385850,0.399616,0.411271,0.420990,...,0.537336,0.552479,0.565089,0.584508,0.603265,0.622829,0.646442,0.687268,0.857458,164.0
22,2008,17007,Boone,159.40,0.297394,0.324646,0.335357,0.344250,0.352041,0.358951,...,0.745452,0.793157,0.815449,0.832058,0.854803,0.878541,0.909917,0.949551,1.092521,169.0
32,2008,17009,Brown,155.60,0.320235,0.344841,0.358400,0.366178,0.376302,0.383874,...,0.318066,0.369261,0.388442,0.416415,0.439445,0.458757,0.485582,0.518587,0.594250,152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986,2023,55133,Waukesha,176.60,0.259277,0.312642,0.334116,0.343132,0.354876,0.360626,...,0.486194,0.555882,0.600575,0.661041,0.722007,0.782591,0.810454,0.904625,1.111393,172.4
10002,2023,55135,Waupaca,160.40,0.300269,0.337200,0.351193,0.361830,0.368585,0.375507,...,0.459658,0.501014,0.544841,0.576299,0.599568,0.622999,0.662759,0.740084,0.813215,162.4
10018,2023,55137,Waushara,169.94,0.298194,0.331473,0.344158,0.353831,0.362061,0.369526,...,0.486593,0.540674,0.576278,0.607568,0.646915,0.677229,0.720712,0.769819,0.877102,179.7
10028,2023,55139,Winnebago,159.16,0.270787,0.315472,0.328980,0.336950,0.346662,0.352519,...,0.776868,0.839917,0.894280,0.941975,0.987939,1.042799,1.105787,1.174166,1.327040,176.1


In [24]:
print(list(final_df_ultimate.columns))
print(len(list(final_df_ultimate.columns)))

['year', 'GEOID', 'NAME', 'Hist_Yield', 'Greenup_1_EVI2_max_p1', 'Greenup_1_EVI2_max_p10', 'Greenup_1_EVI2_max_p20', 'Greenup_1_EVI2_max_p30', 'Greenup_1_EVI2_max_p40', 'Greenup_1_EVI2_max_p50', 'Greenup_1_EVI2_max_p60', 'Greenup_1_EVI2_max_p70', 'Greenup_1_EVI2_max_p80', 'Greenup_1_EVI2_max_p90', 'Greenup_1_EVI2_max_p99', 'Maturity_1_EVI2_max_p1', 'Maturity_1_EVI2_max_p10', 'Maturity_1_EVI2_max_p20', 'Maturity_1_EVI2_max_p30', 'Maturity_1_EVI2_max_p40', 'Maturity_1_EVI2_max_p50', 'Maturity_1_EVI2_max_p60', 'Maturity_1_EVI2_max_p70', 'Maturity_1_EVI2_max_p80', 'Maturity_1_EVI2_max_p90', 'Maturity_1_EVI2_max_p99', 'MidGreendown_1_EVI2_max_p1', 'MidGreendown_1_EVI2_max_p10', 'MidGreendown_1_EVI2_max_p20', 'MidGreendown_1_EVI2_max_p30', 'MidGreendown_1_EVI2_max_p40', 'MidGreendown_1_EVI2_max_p50', 'MidGreendown_1_EVI2_max_p60', 'MidGreendown_1_EVI2_max_p70', 'MidGreendown_1_EVI2_max_p80', 'MidGreendown_1_EVI2_max_p90', 'MidGreendown_1_EVI2_max_p99', 'MidGreenup_1_EVI2_max_p1', 'MidGreenup

In [25]:
# export
final_df_ultimate.reset_index(drop=True, inplace=True)
final_df_ultimate.to_csv(f'/content/drive/MyDrive/Lab/BNN/Data/PBCNN/phenology/corn_input_final.csv', index=False)